# 1. Importing Libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# For statistical analysis
import scipy.stats as stats
from scipy.stats import f_oneway, ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols

# For machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

# For explainability
import shap

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Data Loading

In [ ]:
# Load the dataset from Hugging Face
print("Loading dataset from Hugging Face...")
dataset = load_dataset("lukebarousse/data_jobs")

# Convert to pandas DataFrame
df = pd.DataFrame(dataset['train'])

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Loading dataset from Hugging Face...


README.md: 0.00B [00:00, ?B/s]

data_jobs.csv:   0%|          | 0.00/231M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/785741 [00:00<?, ? examples/s]

Dataset loaded successfully!
Shape: (785741, 17)

First few rows:


,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


In [ ]:
# Explore dataset structure
print(f"\nDataset Shape: {df.shape}")
print(f"Number of Rows: {df.shape[0]:,}")
print(f"Number of Columns: {df.shape[1]}")
print(f"\nColumn Names:")
print(df.columns.tolist())
print(f"\nData Types:")
print(df.dtypes)


Dataset Shape: (785741, 17)
Number of Rows: 785,741
Number of Columns: 17

Column Names:
['job_title_short', 'job_title', 'job_location', 'job_via', 'job_schedule_type', 'job_work_from_home', 'search_location', 'job_posted_date', 'job_no_degree_mention', 'job_health_insurance', 'job_country', 'salary_rate', 'salary_year_avg', 'salary_hour_avg', 'company_name', 'job_skills', 'job_type_skills']

Data Types:
job_title_short           object
job_title                 object
job_location              object
job_via                   object
job_schedule_type         object
job_work_from_home          bool
search_location           object
job_posted_date           object
job_no_degree_mention       bool
job_health_insurance        bool
job_country               object
salary_rate               object
salary_year_avg          float64
salary_hour_avg          float64
company_name              object
job_skills                object
job_type_skills           object
dtype: object


In [ ]:
# Check missing values and data quality
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print(missing_stats.to_string(index=False))

print(df.describe())

           Column  Missing_Count  Missing_Percentage
  salary_hour_avg         775079               98.64
  salary_year_avg         763738               97.20
      salary_rate         752674               95.79
  job_type_skills         117037               14.90
       job_skills         117037               14.90
job_schedule_type          12667                1.61
     job_location           1045                0.13
      job_country             49                0.01
        job_title              1                0.00
          job_via              8                0.00
     company_name             18                0.00
       salary_year_avg  salary_hour_avg
count     22003.000000     10662.000000
mean     123286.274072        47.016598
std       48312.449482        21.890738
min       15000.000000         8.000000
25%       90000.000000        27.500000
50%      115000.000000        45.980000
75%      150000.000000        61.159996
max      960000.000000       391.000000


## 3. Data Preprocessing and Cleaning

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Step 1: Filter for 2023 data only
if 'job_posted_date' in df_processed.columns:
    df_processed['job_posted_date'] = pd.to_datetime(df_processed['job_posted_date'], errors='coerce')
    df_processed['year'] = df_processed['job_posted_date'].dt.year
    df_2023 = df_processed[df_processed['year'] == 2023].copy()
    print(f"Filtered for 2023 data: {len(df_2023):,} rows")
else:
    df_2023 = df_processed.copy()
    print(f"No date filtering applied: {len(df_2023):,} rows")

# Step 2: Handle salary-related columns
# Identify salary columns
salary_columns = [col for col in df_2023.columns if 'salary' in col.lower()]
print(f"\nSalary columns found: {salary_columns}")

# Create unified salary column if multiple salary columns exist
if 'salary_year_avg' in df_2023.columns:
    df_2023['salary'] = df_2023['salary_year_avg']
elif 'salary_yearly' in df_2023.columns:
    df_2023['salary'] = df_2023['salary_yearly']
elif len(salary_columns) > 0:
    df_2023['salary'] = df_2023[salary_columns[0]]
else:
    print("Warning: No salary column found, creating placeholder")
    df_2023['salary'] = np.nan

# Remove rows with missing or zero salaries
df_2023 = df_2023[df_2023['salary'].notna()]
df_2023 = df_2023[df_2023['salary'] > 0]
print(f"After removing missing/zero salaries: {len(df_2023):,} rows")

# Remove outliers (keep salaries between 5th and 95th percentile)
lower_bound = df_2023['salary'].quantile(0.05)
upper_bound = df_2023['salary'].quantile(0.95)
df_2023 = df_2023[(df_2023['salary'] >= lower_bound) & (df_2023['salary'] <= upper_bound)]
print(f"After removing outliers (5th-95th percentile): {len(df_2023):,} rows")
print(f"  Salary range: ${lower_bound:,.0f} - ${upper_bound:,.0f}")

print(f"\nFinal dataset shape: {df_2023.shape}")
print(f"\nSalary Statistics:")
print(df_2023['salary'].describe())

Filtered for 2023 data: 785,741 rows

Salary columns found: ['salary_rate', 'salary_year_avg', 'salary_hour_avg']
After removing missing/zero salaries: 22,003 rows
After removing outliers (5th-95th percentile): 19,896 rows
  Salary range: $57,500 - $203,000

Final dataset shape: (19896, 19)

Salary Statistics:
count     19896.000000
mean     120050.334416
std       34787.340357
min       57500.000000
25%       90000.000000
50%      115000.000000
75%      147500.000000
max      203000.000000
Name: salary, dtype: float64


In [ ]:
# Step 3: Process categorical variables
# Identify key categorical columns
categorical_cols = []

# Remote status
if 'job_work_from_home' in df_2023.columns:
    df_2023['remote_status'] = df_2023['job_work_from_home'].fillna(False).astype(str)
    categorical_cols.append('remote_status')
    print(f"Remote status processed")
    print(f"  Distribution:\n{df_2023['remote_status'].value_counts()}\n")

# Schedule type
if 'job_schedule_type' in df_2023.columns:
    df_2023['schedule_type'] = df_2023['job_schedule_type'].fillna('Unknown')
    categorical_cols.append('schedule_type')
    print(f"Schedule type processed")
    print(f"  Distribution:\n{df_2023['schedule_type'].value_counts()}\n")

# Country
if 'job_country' in df_2023.columns:
    df_2023['country'] = df_2023['job_country'].fillna('Unknown')
    # Keep only top countries (with at least 50 jobs)
    country_counts = df_2023['country'].value_counts()
    top_countries = country_counts[country_counts >= 50].index.tolist()
    df_2023['country'] = df_2023['country'].apply(lambda x: x if x in top_countries else 'Other')
    categorical_cols.append('country')
    print(f"Country processed (keeping countries with ≥50 jobs)")
    print(f"  Distribution:\n{df_2023['country'].value_counts()}\n")

# Job title/role
if 'job_title_short' in df_2023.columns:
    df_2023['job_role'] = df_2023['job_title_short'].fillna('Unknown')
    categorical_cols.append('job_role')
    print(f"Job role processed")
    print(f"  Distribution:\n{df_2023['job_role'].value_counts()}\n")
elif 'job_title' in df_2023.columns:
    df_2023['job_role'] = df_2023['job_title'].fillna('Unknown')
    categorical_cols.append('job_role')

print(f"Total categorical columns identified: {len(categorical_cols)}")
print(f"  Columns: {categorical_cols}")

Remote status processed
  Distribution:
remote_status
False    16919
True      2977
Name: count, dtype: int64

Schedule type processed
  Distribution:
schedule_type
Full-time                                19038
Contractor                                 365
Full-time and Part-time                    283
Part-time                                   56
Full-time and Contractor                    44
Full-time and Temp work                     23
Internship                                  19
Temp work                                   13
Unknown                                     13
Full-time and Internship                     9
Full-time, Part-time, and Internship         8
Contractor and Temp work                     6
Full-time, Contractor, and Temp work         5
Part-time and Contractor                     3
Full-time, Contractor, and Internship        3
Full-time, Part-time, and Contractor         2
Full-time and Per diem                       2
Full-time, Part-time, and Temp work 

In [ ]:
# Step 4: Process skills data
# Find skills columns
skills_cols = [col for col in df_2023.columns if 'skill' in col.lower()]
print(f"Skills columns found: {skills_cols}\n")

# Initialize skills dictionary
all_skills = set()
skills_by_job = []

# Check if there's a job_skills column
if 'job_skills' in df_2023.columns:
    print("Processing job_skills column...")
    for idx, skills in df_2023['job_skills'].items():
        if pd.notna(skills):
            if isinstance(skills, str):
                # Parse string representation of list
                skills_list = eval(skills) if skills.startswith('[') else [s.strip() for s in skills.split(',')]
            elif isinstance(skills, list):
                skills_list = skills
            else:
                skills_list = []

            skills_list = [s.lower().strip() for s in skills_list if s]
            skills_by_job.append(skills_list)
            all_skills.update(skills_list)
        else:
            skills_by_job.append([])

    df_2023['skills_list'] = skills_by_job
    print(f"Skills extracted: {len(all_skills)} unique skills")

    # Create binary columns for common skills (appearing in at least 5% of jobs)
    min_skill_count = len(df_2023) * 0.05
    skill_counts = {}
    for skills in skills_by_job:
        for skill in skills:
            skill_counts[skill] = skill_counts.get(skill, 0) + 1

    common_skills = [skill for skill, count in skill_counts.items() if count >= min_skill_count]
    common_skills = sorted(common_skills, key=lambda x: skill_counts[x], reverse=True)[:30]  # Top 30 skills

    print(f"Common skills (appearing in ≥5% of jobs, top 30): {len(common_skills)}")
    print(f"  Top skills: {common_skills[:10]}")

    # Create binary indicators for each common skill
    for skill in common_skills:
        df_2023[f'skill_{skill}'] = df_2023['skills_list'].apply(lambda x: 1 if skill in x else 0)

    # Count total skills per job
    df_2023['num_skills'] = df_2023['skills_list'].apply(len)
    print(f"\nCreated {len(common_skills)} skill indicator columns")
    print(f"Average skills per job: {df_2023['num_skills'].mean():.1f}")
else:
    print("No job_skills column found. Creating placeholder.")
    df_2023['skills_list'] = [[] for _ in range(len(df_2023))]
    df_2023['num_skills'] = 0
    common_skills = []

print(f"\nSkills per job distribution:")
print(df_2023['num_skills'].describe())

Skills columns found: ['job_skills', 'job_type_skills']

Processing job_skills column...
Skills extracted: 219 unique skills
Common skills (appearing in ≥5% of jobs, top 30): 26
  Top skills: ['sql', 'python', 'r', 'aws', 'tableau', 'spark', 'excel', 'azure', 'sas', 'java']

Created 26 skill indicator columns
Average skills per job: 5.5

Skills per job distribution:
count    19896.000000
mean         5.480147
std          4.037984
min          0.000000
25%          3.000000
50%          5.000000
75%          8.000000
max         40.000000
Name: num_skills, dtype: float64
